# ERDDAP with Altair +Argos 

- using erddap data service
- using Altair visual graphics service
- specifically for connecting to drifters (but met data will likely be possible too)

TODO:


In [1]:
from erddapy import ERDDAP
import pandas as pd

import altair as alt
### own colormap
import palettable
from vega_datasets import data

In [2]:
server_url = 'http://downdraft.pmel.noaa.gov:8080/erddap'

year = 2020

In [3]:
e = ERDDAP(server=server_url)
df = pd.read_csv(e.get_search_url(response='csv', search_for='Argos drifter NRT'))
print(df['Dataset ID'].values)

['2015_Argos_Drifters_NRT' '2016_Argos_Drifters_NRT'
 '2017_Argos_Drifters_NRT' '2018_Argos_Drifters_NRT'
 '2019_Argos_Drifters_NRT' '2020_Argos_Drifters_NRT']


In [4]:
from requests.exceptions import HTTPError

dfs = {}
for arfloat in df['Dataset ID'].values:
    print(arfloat)
    try:
        e = ERDDAP(server=server_url,
            protocol='tabledap',
            response='csv'
        )
        e.dataset_id=arfloat
    except HTTPError:
        print('Failed to generate url {}'.format(alamo))
        continue
    try:
        df = e.to_pandas(
            index_col='time (UTC)',
            parse_dates=True,
            skiprows=(1,)  # units information can be dropped.
       ).dropna()
        
        df.sort_index(inplace=True)
        df.columns = [x[1].split()[0] for x in enumerate(df.columns)]

        dfs.update({arfloat: df})
    except:
        print("error")
        pass

2015_Argos_Drifters_NRT
2016_Argos_Drifters_NRT
2017_Argos_Drifters_NRT
2018_Argos_Drifters_NRT
2019_Argos_Drifters_NRT
2020_Argos_Drifters_NRT


In [5]:
dft = pd.DataFrame()
for dfn, df in dfs.items():
    dft = pd.concat([dft,df])
    

In [6]:
dft

,trajectory_id,strain,voltage,latitude,sst,longitude
time (UTC),,,,,,
2014-12-31 21:37:00+00:00,122533,1.0,11.6,56.147,7.56,-142.728
2014-12-31 21:38:00+00:00,122843,0.0,12.2,56.356,3.80,-160.384
2014-12-31 21:38:00+00:00,136871,16.0,13.8,58.857,4.04,-172.819
2014-12-31 21:38:00+00:00,136874,26.0,14.8,55.354,4.28,-178.985
2014-12-31 21:43:00+00:00,122846,12.0,12.4,57.903,3.52,-166.517
...,...,...,...,...,...,...
2020-06-25 09:24:00+00:00,136863,1.0,13.0,56.361,10.96,163.331
2020-06-25 09:45:00+00:00,136863,1.0,13.0,56.360,10.80,163.329
2020-06-25 18:44:00+00:00,136863,1.0,13.0,56.363,8.64,163.336


In [7]:
alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

data.world_110m.url   
or    
https://cdn.jsdelivr.net/npm/world-atlas@2/land-50m.json

https://cdn.jsdelivr.net/npm/world-atlas@2/land-50m.json

In [8]:
# Data generators for the background
sphere = alt.sphere()
graticule = alt.graticule()

# Source of land data
source = alt.topo_feature('https://cdn.jsdelivr.net/npm/world-atlas@2/land-50m.json', 'land')

# Layering and configuring the components
base = alt.layer(
    alt.Chart(sphere).mark_geoshape(fill='White'),
    alt.Chart(graticule).mark_geoshape(stroke='grey', strokeWidth=0.5),
    alt.Chart(source).mark_geoshape(fill='White', stroke='black')
).project(
    type='azimuthalEqualArea',
    rotate=[160,-90,0],
    scale=750,
    center=[0,-30]
).properties(width=600, height=400).configure_view(stroke=None)

base

alt.LayerChart(...)

In [10]:
year = '2020'
points = alt.Chart(dft[year].reset_index()).mark_point(size=2).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='trajectory_id:N'
)

(base+points)

alt.LayerChart(...)

In [11]:
ts = alt.Chart(dft[year].reset_index()).mark_line().encode(
    alt.X('time (UTC):T'),
    alt.Y('sst:Q'),
    alt.Color('trajectory_id:N')
).properties(
    width=750,
    height=150
).interactive()
ts

alt.Chart(...)